In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from scipy import stats
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.calibration import CalibratedClassifierCV
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv('test.csv')

In [83]:
X_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
4,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1


In [84]:
X_train[X_train['target']==1]['text']

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       #RockyFire Update => California Hwy. 20 closed...
4       I'm on top of the hill and I can see a fire in...
                              ...                        
5341    Suicide bomber kills 15 in Saudi security site...
5342    Two giant cranes holding a bridge collapse int...
5343    @aria_ahrary @TheTawniest The out of control w...
5344    Police investigating after an e-bike collided ...
5345    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 2291, dtype: object

In [85]:
X_train[X_train['target']==0]['text']

8                                          What's up man?
9                                           I love fruits
10                                       Summer is lovely
11                                      My car is so fast
12                                 this is ridiculous....
                              ...                        
5325    @widda16 ... He's gone. You can relax. I thoug...
5326     @jt_ruff23 @cameronhacker and I wrecked you both
5327    Three days off from work and they've pretty mu...
5328    @engineshed Great atmosphere at the British Li...
5329    Cramer: Iger's 3 words that wrecked Disney's s...
Name: text, Length: 3055, dtype: object

In [86]:
print('X_train.shape(sin procesar): ', X_train.shape)
print('X_test.shape(sin procesar): ', X_test.shape)

X_train.shape(sin procesar):  (5346, 5)
X_test.shape(sin procesar):  (2267, 4)


In [87]:
# Eliminamos los duplicados
X_train = X_train.drop_duplicates(subset='text')
print('X_train.shape(sin duplicados): ', X_train.shape)

X_train.shape(sin duplicados):  (5286, 5)


In [88]:
import re

signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\>)|(\=)|(\<)")
signos_arroba = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\>)|(\=)|(\<)|(\@)")

def signs_tweets(tweet):
    return signos.sub('', tweet.lower())

X_train['text'] = X_train['text'].apply(signs_tweets)
X_train['text'].head()
X_test['text'] = X_test['text'].apply(signs_tweets)
X_test['text'].head()

0     people receive #wildfires evacuation orders i...
1    just got sent this photo from ruby #alaska as ...
2    #flood #disaster heavy rain causes flash flood...
3    there's an emergency evacuation happening now ...
4    i'm afraid that the tornado is coming to our area
Name: text, dtype: object

In [89]:
def remove_links(df):
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

X_train['text'] = X_train['text'].apply(remove_links)
X_test['text'] = X_test['text'].apply(remove_links)

In [90]:
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

def remove_stopwords(df):
    return " ".join([word for word in df.split() if word not in english_stopwords])

X_train['text'] = X_train['text'].apply(remove_stopwords)
X_train.head()
X_test['text'] = X_test['text'].apply(remove_stopwords)
X_test.head()

,id,keyword,location,text
0,6,NaN,NaN,people receive #wildfires evacuation orders ca...
1,7,NaN,NaN,got sent photo ruby #alaska smoke #wildfires p...
2,10,NaN,NaN,#flood #disaster heavy rain causes flash flood...
3,14,NaN,NaN,there's emergency evacuation happening buildin...
4,15,NaN,NaN,i'm afraid tornado coming area


In [91]:
from nltk.stem.snowball import SnowballStemmer

def english_stemmer(x):
    stemmer = SnowballStemmer('english')
    return ' '.join([stemmer.stem(word) for word in x.split()])

X_train['text'] = X_train['text'].apply(english_stemmer)
X_test['text'] = X_test['text'].apply(english_stemmer)

In [92]:
X_train = X_train[['text', 'target']]

In [93]:
"""vectorizer = CountVectorizer(binary=True)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)"""
vectorizer = CountVectorizer()

In [94]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('cls', SVC(probability=True))
])

In [151]:
# Aqui definimos el espacio de parámetros a explorar
parameters = {
    'vect__max_df': (0.5, 1.9),
    'vect__min_df': (10, 20,50),
    'vect__max_features': (500, 1000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigramas or bigramas
    'cls__C': (0.2, 0.5, 0.7),
    'cls__degree': [2,3,4,5],
    'cls__gamma': [0.001, 0.1, "auto", 1.0, 10.0, 30.0]
}

parameters_low = {
    'vect__max_df': (0.05, 0.06),
    'vect__min_df': (0.05, 0.04),
    'vect__max_features': (550, 740),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigramas or bigramas
    'cls__C': (0.07, 0.1),
    'cls__degree': [0.1, 0.2],
    'cls__gamma': ['auto', 0.1]
}


grid_search = GridSearchCV(pipeline,
                          parameters_low,
                          cv=5,
                          n_jobs=-2,
                          scoring='roc_auc')

In [152]:
grid_search.fit(X_train['text'], X_train['target'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('cls', SVC(probability=True))]),
             n_jobs=-2,
             param_grid={'cls__C': (0.07, 0.1), 'cls__degree': [0.1, 0.2],
                         'cls__gamma': ['auto', 0.1],
                         'vect__max_df': (0.05, 0.06),
                         'vect__max_features': (550, 740),
                         'vect__min_df': (0.05, 0.04),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             scoring='roc_auc')

In [153]:
print("Best params:", grid_search.best_params_)
print("Best acc:", grid_search.best_score_)
print("Best model:", grid_search.best_estimator_)

Best params: {'cls__C': 0.07, 'cls__degree': 0.1, 'cls__gamma': 'auto', 'vect__max_df': 0.06, 'vect__max_features': 550, 'vect__min_df': 0.04, 'vect__ngram_range': (1, 1)}
Best acc: 0.5201917199073144
Best model: Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.06, max_features=550, min_df=0.04)),
                ('cls',
                 SVC(C=0.07, degree=0.1, gamma='auto', probability=True))])


In [154]:
grid_search.predict(X_test['text'])

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [155]:
predict_proba = grid_search.predict_proba(X_test['text'])

In [156]:
predict_proba

array([[0.58746351, 0.41253649],
       [0.58746351, 0.41253649],
       [0.58746351, 0.41253649],
       ...,
       [0.58746351, 0.41253649],
       [0.58746351, 0.41253649],
       [0.58746351, 0.41253649]])

In [157]:
predict_proba.shape

(2267, 2)

In [158]:
df_submission = X_test[['id']]
df_submission['target'] = predict_proba[:,-1]
df_submission.to_csv('data/submission.csv', index=False)

<ipython-input-158-2328126d8665>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submission['target'] = predict_proba[:,-1]


In [72]:
from sklearn.feature_extraction.text import CountVectorizer

baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(X_train)

X_baseline = baseline_vectorizer.transform(X_train)
X_test_baseline = baseline_vectorizer.transform()

In [73]:
from sklearn.linear_model import LogisticRegression

In [74]:
logisticRegr = LogisticRegression()

logisticRegr.fit(X_train['text'], X_train['target'])

IndexError: Index dimension must be <= 2